# Urban Flood Vulnerability Assessment - Colombo District, Sri Lanka

**Assignment 2 - Scientific Programming for Geospatial Sciences**

**Authors:** Surya Jamuna Rani Subramaniyan (S3664414) & Sachin Ravi (S3563545)

---

## Contents

0. **Setup & Data Download** - Automated data acquisition
1. **Data Loading** - Load and preprocess datasets
2. **NumPy Array Operations** - Raster processing
3. **PyTorch Tensor Operations** - GPU-aware processing with performance comparison
4. **Vector Processing** - GeoPandas/Shapely operations (3+)
5. **Xarray Data Cubes** - Multi-temporal analysis
6. **Raster-Vector Integration** - Bidirectional operations
7. **Visualization** - Maps and dashboard

---

**Study Area:** Colombo District, Sri Lanka  
**Bounding Box:** 79.82°E - 80.22°E, 6.75°N - 7.05°N

## 0. Setup & Data Download

Run this section once to download all required datasets automatically.  
**No API keys required!**

In [ ]:
# imports
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import torch
import matplotlib.pyplot as plt
import requests
import json
import zipfile
import gzip
import shutil
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# our modules
import sys
sys.path.append('..')
from src import data_loading, raster_analysis, tensor_operations, vector_analysis, integration, visualization

print("✅ All imports successful!")

In [ ]:
# Study area configuration
COLOMBO_BBOX = {
    'west': 79.82,
    'east': 80.22,
    'south': 6.75,
    'north': 7.05
}

# Paths
DATA_DIR = Path('../data')
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUT_DIR = Path('../outputs')

# Create directories
for d in [RAW_DIR / 'chirps', RAW_DIR / 'dem', RAW_DIR / 'admin', 
          RAW_DIR / 'buildings', PROCESSED_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("📁 Directories created:")
print(f"   - {RAW_DIR}")
print(f"   - {PROCESSED_DIR}")
print(f"   - {OUTPUT_DIR}")

In [ ]:
# Helper function for downloads
def download_file(url, output_path, timeout=60):
    """Download a file with progress indication."""
    if output_path.exists():
        print(f"  ✅ Already exists: {output_path.name}")
        return True
    
    try:
        print(f"  ⬇️ Downloading: {output_path.name}")
        response = requests.get(url, stream=True, timeout=timeout)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"  ✅ Saved: {output_path.name}")
        return True
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

### 0.1 Download SRTM DEM from AWS

In [ ]:
# Download SRTM tiles from AWS Open Data (no API key needed!)
print("📡 Downloading SRTM DEM from AWS...")

# Get required tiles for Colombo
srtm_tiles = ['N06E079', 'N06E080', 'N07E079', 'N07E080']
base_url = "https://elevation-tiles-prod.s3.amazonaws.com/skadi"

dem_files = []
for tile in srtm_tiles:
    lat_dir = tile[:3]
    filename = f"{tile}.hgt.gz"
    url = f"{base_url}/{lat_dir}/{filename}"
    output_path = RAW_DIR / 'dem' / filename
    
    if download_file(url, output_path, timeout=120):
        dem_files.append(output_path)

print(f"\n✅ Downloaded {len(dem_files)} DEM tiles")

### 0.2 Download Administrative Boundaries from OSM

In [ ]:
# Download Colombo District boundary using Overpass API
print("📡 Downloading admin boundaries from OpenStreetMap...")

admin_path = RAW_DIR / 'admin' / 'colombo_boundary.json'

if not admin_path.exists():
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    # Query for Colombo District and its subdivisions
    query = f"""
    [out:json][timeout:120];
    (
      relation["name"~"Colombo"]["admin_level"~"5|6|7"]
        ({COLOMBO_BBOX['south']},{COLOMBO_BBOX['west']},{COLOMBO_BBOX['north']},{COLOMBO_BBOX['east']});
    );
    out geom;
    """
    
    try:
        response = requests.post(overpass_url, data={'data': query}, timeout=180)
        response.raise_for_status()
        data = response.json()
        
        with open(admin_path, 'w') as f:
            json.dump(data, f)
        
        print(f"  ✅ Saved: {admin_path.name}")
        print(f"  Found {len(data.get('elements', []))} boundary elements")
    except Exception as e:
        print(f"  ❌ Error: {e}")
else:
    print(f"  ✅ Already exists: {admin_path.name}")

### 0.3 Download CHIRPS Rainfall Data

⚠️ **Note:** CHIRPS files are large (~2GB per year). For testing, we'll use a sample.

In [ ]:
# CHIRPS download - for demo we'll use sample data
# Uncomment below to download actual CHIRPS (WARNING: ~2GB per year)

print("📡 CHIRPS Rainfall Data")
print("")
print("For full analysis, download CHIRPS data manually:")
print("  URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/")
print("  File: chirps-v2.0.2023.days_p05.nc (~2GB)")
print("  Save to: data/raw/chirps/")
print("")
print("For this demo, we'll generate sample rainfall data.")

# To download automatically (uncomment if needed):
# chirps_url = "https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/chirps-v2.0.2023.days_p05.nc"
# chirps_path = RAW_DIR / 'chirps' / 'chirps-v2.0.2023.days_p05.nc'
# download_file(chirps_url, chirps_path, timeout=3600)  # may take a while!

### 0.4 Download Buildings (Optional)

Buildings can come from Google Open Buildings or OSM.

In [ ]:
# Download OSM buildings (optional - can be slow for urban areas)
print("📡 Building Footprints")
print("")
print("Options:")
print("  1. Google Open Buildings: https://sites.research.google/open-buildings/")
print("  2. OSM buildings via Overpass (can be slow)")
print("")
print("For this demo, we'll generate sample building data.")

# To download OSM buildings (uncomment if needed - may take 5-10 minutes):
# buildings_query = f"""
# [out:json][timeout:300];
# (way["building"]({COLOMBO_BBOX['south']},{COLOMBO_BBOX['west']},{COLOMBO_BBOX['north']},{COLOMBO_BBOX['east']}););
# out geom;
# """
# response = requests.post("https://overpass-api.de/api/interpreter", data={'data': buildings_query}, timeout=600)
# with open(RAW_DIR / 'buildings' / 'osm_buildings.json', 'w') as f:
#     json.dump(response.json(), f)

### 0.5 Data Status Check

In [ ]:
# Check what data we have
print("📊 DATA STATUS")
print("=" * 50)

# DEM
dem_files_found = list((RAW_DIR / 'dem').glob('*.hgt.gz'))
print(f"DEM tiles:      {'✅ ' + str(len(dem_files_found)) + ' files' if dem_files_found else '❌ Not found'}")

# Admin boundaries
admin_files = list((RAW_DIR / 'admin').glob('*.json'))
print(f"Admin boundary: {'✅ Found' if admin_files else '❌ Not found'}")

# CHIRPS
chirps_files = list((RAW_DIR / 'chirps').glob('*.nc'))
print(f"CHIRPS data:    {'✅ Found' if chirps_files else '⚠️ Using sample data'}")

# Buildings
building_files = list((RAW_DIR / 'buildings').glob('*'))
print(f"Buildings:      {'✅ Found' if building_files else '⚠️ Using sample data'}")

print("=" * 50)
print("\n🚀 Ready to proceed with analysis!")

---

## 1. Data Loading

Load the required datasets for analysis.

In [ ]:
# For this demo, we create sample data
# When you have real data, use data_loading module functions

print("📊 Creating sample datasets for demonstration...")

# Sample rainfall data (simulating CHIRPS)
np.random.seed(42)
times = pd.date_range('2023-01-01', periods=365, freq='D')
lats = np.linspace(COLOMBO_BBOX['south'], COLOMBO_BBOX['north'], 50)
lons = np.linspace(COLOMBO_BBOX['west'], COLOMBO_BBOX['east'], 50)

# Create realistic rainfall patterns (monsoon effect)
rainfall_data = np.random.exponential(scale=15, size=(365, 50, 50))
# Add monsoon peak (May-September)
rainfall_data[120:270] *= 2.5

rainfall_cube = xr.DataArray(
    data=rainfall_data,
    dims=['time', 'latitude', 'longitude'],
    coords={'time': times, 'latitude': lats, 'longitude': lons},
    name='precipitation'
)

print(f"✅ Rainfall data: {rainfall_cube.shape}")
print(f"   Time range: {rainfall_cube.time.min().values} to {rainfall_cube.time.max().values}")

In [ ]:
# Sample elevation data (simulating DEM)
np.random.seed(42)
# Colombo is coastal - elevation increases inland (east)
x = np.linspace(0, 1, 50)
y = np.linspace(0, 1, 50)
X, Y = np.meshgrid(x, y)
elevation_data = (X * 50 + np.random.normal(0, 5, (50, 50))).clip(0, 100)  # 0-100m

elevation = xr.DataArray(
    data=elevation_data,
    dims=['latitude', 'longitude'],
    coords={'latitude': lats, 'longitude': lons},
    name='elevation'
)

print(f"✅ Elevation data: {elevation.shape}")
print(f"   Range: {elevation.min().values:.1f}m - {elevation.max().values:.1f}m")

In [ ]:
# Sample admin boundaries and buildings
from shapely.geometry import box, Point

# Create Colombo sub-districts (Divisional Secretariats)
ds_names = ['Colombo', 'Thimbirigasyaya', 'Dehiwala', 'Moratuwa', 'Sri Jayawardenepura Kotte']

admin_boundaries = gpd.GeoDataFrame({
    'ds_id': [f'DS{i+1:02d}' for i in range(5)],
    'ds_name': ds_names,
    'geometry': [
        box(79.82, 6.90, 79.90, 7.00),
        box(79.90, 6.90, 79.98, 7.00),
        box(79.82, 6.82, 79.90, 6.90),
        box(79.82, 6.75, 79.90, 6.82),
        box(79.98, 6.90, 80.10, 7.00)
    ]
}, crs='EPSG:4326')

print(f"✅ Admin boundaries: {len(admin_boundaries)} divisions")
print(admin_boundaries[['ds_id', 'ds_name']])

In [ ]:
# Sample buildings (more in urban core)
np.random.seed(42)

# Generate more buildings in western (coastal/urban) areas
n_buildings = 500
building_lons = np.random.beta(2, 5, n_buildings) * (COLOMBO_BBOX['east'] - COLOMBO_BBOX['west']) + COLOMBO_BBOX['west']
building_lats = np.random.uniform(COLOMBO_BBOX['south'], COLOMBO_BBOX['north'], n_buildings)

buildings = gpd.GeoDataFrame({
    'building_id': [f'B{i:04d}' for i in range(n_buildings)],
    'geometry': [Point(lon, lat).buffer(0.0005) for lon, lat in zip(building_lons, building_lats)]
}, crs='EPSG:4326')

print(f"✅ Buildings: {len(buildings)} footprints")

In [ ]:
# Visualize the study area
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Rainfall
rainfall_cube.mean(dim='time').plot(ax=axes[0], cmap='Blues')
axes[0].set_title('Mean Daily Rainfall (mm)')

# Elevation
elevation.plot(ax=axes[1], cmap='terrain')
axes[1].set_title('Elevation (m)')

# Admin and buildings
admin_boundaries.plot(ax=axes[2], alpha=0.3, edgecolor='black')
buildings.plot(ax=axes[2], color='red', markersize=1, alpha=0.5)
axes[2].set_title('Admin Boundaries & Buildings')

plt.tight_layout()
plt.show()

---

## 2. NumPy Array Operations

Demonstrate array-based raster processing.

In [ ]:
# Get rainfall as numpy array
rainfall_np = rainfall_cube.values
print(f"Rainfall array shape: {rainfall_np.shape} (days, lat, lon)")

In [ ]:
# Operation 1: Create extreme rainfall mask (>50mm threshold)
extreme_mask = raster_analysis.create_extreme_rainfall_mask(rainfall_np, threshold=50)
print(f"Extreme rainfall events (>50mm): {extreme_mask.sum()} occurrences")

In [ ]:
# Operation 2: Count extreme events per location
extreme_counts = raster_analysis.count_extreme_events(rainfall_np, threshold=50)
print(f"Max extreme events at any location: {extreme_counts.max()}")

plt.figure(figsize=(8, 6))
plt.imshow(extreme_counts, cmap='Reds', origin='lower',
           extent=[COLOMBO_BBOX['west'], COLOMBO_BBOX['east'], 
                   COLOMBO_BBOX['south'], COLOMBO_BBOX['north']])
plt.colorbar(label='Number of extreme rainfall days')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Extreme Rainfall Events (>50mm) in 2023')
plt.show()

In [ ]:
# Operation 3: Calculate 95th percentile rainfall
p95_rainfall = raster_analysis.calculate_percentile_rainfall(rainfall_np, percentile=95)
print(f"95th percentile range: {p95_rainfall.min():.2f} - {p95_rainfall.max():.2f} mm")

In [ ]:
# Operation 4: Normalize for vulnerability calculation
rainfall_norm = raster_analysis.normalize_array(p95_rainfall, method='minmax')
elevation_norm = raster_analysis.normalize_array(elevation.values, method='minmax')

print(f"Normalized rainfall range: {rainfall_norm.min():.3f} - {rainfall_norm.max():.3f}")
print(f"Normalized elevation range: {elevation_norm.min():.3f} - {elevation_norm.max():.3f}")

---

## 3. PyTorch Tensor Operations

GPU-aware processing with performance comparison.

In [ ]:
# Check GPU availability
tensor_operations.print_gpu_info()

In [ ]:
# Convert to tensor
rainfall_tensor = tensor_operations.numpy_to_tensor(p95_rainfall, device='auto')
print(f"Tensor device: {rainfall_tensor.device}")
print(f"Tensor shape: {rainfall_tensor.shape}")

In [ ]:
# Apply Gaussian convolution for spatial smoothing
smoothed_tensor = tensor_operations.apply_gaussian_convolution(
    rainfall_tensor, kernel_size=5, sigma=1.5
)

# Convert back to numpy for visualization
smoothed = tensor_operations.tensor_to_numpy(smoothed_tensor)

# Compare
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(p95_rainfall, cmap='Blues', origin='lower')
axes[0].set_title('Original 95th Percentile Rainfall')
axes[1].imshow(smoothed, cmap='Blues', origin='lower')
axes[1].set_title('Smoothed (PyTorch Gaussian Convolution)')
plt.tight_layout()
plt.show()

In [ ]:
# PERFORMANCE COMPARISON: NumPy vs PyTorch
print("🔬 Running performance comparison...")
print("(This measures Gaussian convolution speed)\n")

perf_results = tensor_operations.compare_numpy_vs_torch(
    p95_rainfall, kernel_size=5, sigma=1.5, num_iterations=10
)

print("=" * 55)
print("        PERFORMANCE COMPARISON RESULTS")
print("=" * 55)
print(f"Array size:        {p95_rainfall.shape}")
print(f"Operation:         5x5 Gaussian Convolution")
print(f"Iterations:        10")
print("")
print(f"NumPy (scipy):     {perf_results['numpy_time']*1000:.2f} ms ± {perf_results['numpy_std']*1000:.2f} ms")
print(f"PyTorch ({perf_results['device']:6s}):  {perf_results['torch_time']*1000:.2f} ms ± {perf_results['torch_std']*1000:.2f} ms")
print("")
print(f"Speedup:           {perf_results['speedup']:.2f}x")
print("=" * 55)

---

## 4. Vector Processing (GeoPandas/Shapely)

At least 3 vector operations as required.

In [ ]:
# OPERATION 1: Spatial Join - assign DS to each building
buildings_joined = vector_analysis.spatial_join_buildings_to_admin(
    buildings, admin_boundaries, admin_id_col='ds_id'
)

print("📍 OPERATION 1: Spatial Join")
print(f"   Buildings with DS assignment: {len(buildings_joined)}")
print(buildings_joined[['building_id', 'ds_id']].head())

In [ ]:
# OPERATION 2: Building Density Calculation
admin_with_density = vector_analysis.calculate_building_density(
    buildings, admin_boundaries, admin_id_col='ds_id'
)

print("📍 OPERATION 2: Building Density Calculation")
print(admin_with_density[['ds_name', 'building_count', 'building_density']])

In [ ]:
# OPERATION 3: Centroid calculation and area
from shapely.geometry import LineString

# Create sample roads
roads = gpd.GeoDataFrame({
    'road_id': ['R01', 'R02', 'R03'],
    'highway': ['primary', 'secondary', 'primary'],
    'geometry': [
        LineString([(79.82, 6.9), (80.1, 6.9)]),
        LineString([(79.9, 6.75), (79.9, 7.0)]),
        LineString([(79.85, 6.85), (80.0, 6.95)])
    ]
}, crs='EPSG:4326')

# Buffer analysis
road_buffers = vector_analysis.create_road_buffers(
    roads, buffer_distance=0.005, road_types=['primary', 'secondary']
)

print("📍 OPERATION 3: Buffer Analysis")
print(f"   Created {len(road_buffers)} road buffers")

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
admin_with_density.plot(ax=ax, column='building_density', cmap='Reds', alpha=0.5, legend=True)
road_buffers.plot(ax=ax, color='yellow', alpha=0.5)
roads.plot(ax=ax, color='black', linewidth=2)
buildings.plot(ax=ax, color='blue', markersize=1, alpha=0.3)
ax.set_title('Building Density & Road Infrastructure')
plt.show()

---

## 5. Xarray Data Cubes

Multi-temporal analysis.

In [ ]:
# Show data cube structure
print("📊 Rainfall Data Cube Structure:")
print(rainfall_cube)

In [ ]:
# Monthly aggregation
monthly_mean = rainfall_cube.groupby('time.month').mean(dim='time')

# Plot monthly pattern
monthly_spatial_mean = monthly_mean.mean(dim=['latitude', 'longitude'])

plt.figure(figsize=(10, 4))
monthly_spatial_mean.plot(marker='o')
plt.xlabel('Month')
plt.ylabel('Mean Daily Rainfall (mm)')
plt.title('Seasonal Rainfall Pattern - Colombo District')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Monsoon analysis (May-September)
monsoon = rainfall_cube.sel(time=slice('2023-05-01', '2023-09-30'))
non_monsoon = rainfall_cube.sel(time=slice('2023-01-01', '2023-04-30'))

print(f"Monsoon mean:     {monsoon.mean().values:.2f} mm/day")
print(f"Non-monsoon mean: {non_monsoon.mean().values:.2f} mm/day")
print(f"Monsoon ratio:    {monsoon.mean().values / non_monsoon.mean().values:.2f}x")

---

## 6. Raster-Vector Integration

Bidirectional integration as required.

In [ ]:
# Save rasters for integration
import rasterio
from rasterio.transform import from_bounds

bounds = (COLOMBO_BBOX['west'], COLOMBO_BBOX['south'], 
          COLOMBO_BBOX['east'], COLOMBO_BBOX['north'])

# Save rainfall raster
rainfall_path = OUTPUT_DIR / 'p95_rainfall.tif'
with rasterio.open(
    rainfall_path, 'w', driver='GTiff',
    height=50, width=50, count=1, dtype='float32',
    crs='EPSG:4326', transform=from_bounds(*bounds, 50, 50)
) as dst:
    dst.write(p95_rainfall.astype('float32'), 1)
print(f"✅ Saved: {rainfall_path}")

# Save elevation raster
elevation_path = OUTPUT_DIR / 'elevation.tif'
with rasterio.open(
    elevation_path, 'w', driver='GTiff',
    height=50, width=50, count=1, dtype='float32',
    crs='EPSG:4326', transform=from_bounds(*bounds, 50, 50)
) as dst:
    dst.write(elevation.values.astype('float32'), 1)
print(f"✅ Saved: {elevation_path}")

In [ ]:
# RASTER → VECTOR: Zonal Statistics
admin_with_rainfall = integration.extract_zonal_statistics(
    admin_boundaries, rainfall_path,
    stats=['mean', 'max'], prefix='rainfall_'
)

admin_with_elev = integration.extract_zonal_statistics(
    admin_with_rainfall, elevation_path,
    stats=['mean', 'min'], prefix='elevation_'
)

print("📍 RASTER → VECTOR: Zonal Statistics")
print(admin_with_elev[['ds_name', 'rainfall_mean', 'elevation_mean']])

In [ ]:
# VECTOR → RASTER: Rasterize building density
density_raster = integration.rasterize_vector(
    admin_with_density,
    value_column='building_density',
    resolution=(-0.005, 0.005)
)

print("📍 VECTOR → RASTER: Rasterized Building Density")
print(f"   Shape: {density_raster.shape}")

plt.figure(figsize=(8, 6))
density_raster.plot(cmap='Oranges')
plt.title('Rasterized Building Density')
plt.show()

### Calculate Vulnerability Index

In [ ]:
# Combine all factors for vulnerability
result = admin_with_density.merge(
    admin_with_elev[['ds_id', 'rainfall_mean', 'rainfall_max', 'elevation_mean', 'elevation_min']],
    on='ds_id'
)

# Normalize factors
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-10)

rainfall_norm = normalize(result['rainfall_mean'])
density_norm = normalize(result['building_density'])
elev_norm = normalize(result['elevation_mean'])

# Calculate vulnerability: V = 0.4*rainfall + 0.3*density + 0.3*(1-elevation)
result['vulnerability_score'] = (
    0.4 * rainfall_norm +
    0.3 * density_norm +
    0.3 * (1 - elev_norm)  # low elevation = high vulnerability
)

# Classify
result['vulnerability_class'] = pd.cut(
    result['vulnerability_score'],
    bins=[0, 0.3, 0.5, 0.7, 1.0],
    labels=['Low', 'Moderate', 'High', 'Extreme']
)

print("📊 VULNERABILITY ASSESSMENT RESULTS")
print("=" * 60)
print(result[['ds_name', 'rainfall_mean', 'building_density', 'elevation_mean', 'vulnerability_score', 'vulnerability_class']])
print("=" * 60)

---

## 7. Visualization

Final maps and outputs.

In [ ]:
# Add required columns for visualization
result['id'] = result['ds_id']

# Create interactive vulnerability map
vuln_map = visualization.create_vulnerability_map(
    result,
    value_column='vulnerability_score',
    title='Flood Vulnerability Score'
)

# Display
vuln_map

In [ ]:
# Save interactive map
vuln_map.save(OUTPUT_DIR / 'vulnerability_map.html')
print(f"✅ Saved: {OUTPUT_DIR / 'vulnerability_map.html'}")

In [ ]:
# Create ranking chart
ranking_chart = visualization.create_vulnerability_ranking_chart(
    result,
    name_column='ds_name',
    value_column='vulnerability_score',
    top_n=10,
    title='Vulnerability Ranking - Colombo District'
)
ranking_chart.show()

In [ ]:
# Create static map for report
fig = visualization.create_static_map(
    result,
    value_column='vulnerability_score',
    title='Flood Vulnerability Assessment - Colombo District, Sri Lanka',
    cmap='YlOrRd'
)

fig.savefig(OUTPUT_DIR / 'vulnerability_map.png', dpi=150, bbox_inches='tight')
print(f"✅ Saved: {OUTPUT_DIR / 'vulnerability_map.png'}")

---

## Summary

This notebook demonstrated all required technical components:

| Component | Implementation |
|-----------|----------------|
| **NumPy Arrays** | Masking, normalization, percentile calculation |
| **PyTorch Tensors** | Gaussian convolution, GPU-awareness, performance comparison |
| **Vector Processing** | Spatial join, density calculation, buffer analysis (3+ ops) |
| **Xarray Data Cubes** | Temporal slicing, aggregation, groupby operations |
| **Raster-Vector Integration** | Zonal statistics (R→V), rasterization (V→R) |

**Vulnerability Formula:**

$$V = 0.4 \times Rainfall_{norm} + 0.3 \times BuildingDensity_{norm} + 0.3 \times (1 - Elevation_{norm})$$